# 샘플 데이터 생성  
연습에 사용할 샘플 데이터를 생성합니다.  
- 채널/시간/주중말 인벤 : table-chn, 200 * 24 * 2.  
- 셋탑/지역 인벤 : table-set, 1,000 만.  
- 셋탑/타겟 매핑 정보 : table-map  
- 타겟 인벤 : table-seg, 200.    
  
inventory db 에 생성.  
연산 전에 hive 에서 생성해서 넘겨주는 데이터  
> 6개월 데이터로 컬럼 확장해 본다 

In [1]:
# 필요 라이브러리 임포트  
import socket
import sys
import os
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.sql import Row, SparkSession
from os.path import abspath
import findspark
import time 

In [2]:
# 환경변수 정의  
scale = 1000 # 1000 만 건 수준
PRJ_ROOT = '/user/root'
APP_NAME = f'spark-01-sample-creation-{scale}'
DB_NAME = 'inven'

In [3]:
# 스파크 생성 
def spark_creation():
    spark = SparkSession.builder.master('yarn').appName(APP_NAME)\
    .config('spark.driver.cores', '2').config('spark.driver.memory', '4g')\
    .config('spark.num.executors', '3')\
    .config('spark.executor.cores', '2').config('spark.executor.memory', '2g').getOrCreate()
    sc = spark.sparkContext
    sc
    return spark

In [4]:
spark = spark_creation()

/usr/local/lib/python3.6/dist-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [5]:
# 샘플 데이터 생성 
def create_setops():
    setop_count = scale * 10000
    inv_rate = 1.0 
    inv_val = 1000
    setop_name = ['ST_A', 'ST_B', 'ST_C', 'ST_D', 'ST_E', 'ST_F', 'ST_G', 'ST_H', 'ST_I', 'ST_J']
    setops = []
    for s in setop_name:
        for i in range(0, int(setop_count/len(setop_name))):
            setops.append([f'{s}_{i:07d}', inv_rate, inv_val])
            
    print(setops[-10:])
    return setops
# 샘플 데이터 형식 정의. 읽기/쓰기 편의 제공. 
def define_schema():
    from pyspark.sql.types import StructType, StructField, StringType, LongType, FloatType
    columns = [
        StructField("setop", StringType())
        , StructField("inv_rate", FloatType())
        , StructField("inv_val", LongType())
    ]
    sample_schema = StructType(columns)
    return sample_schema

In [6]:
%%time 
# 샘플 데이터 생성 및 확인 
sample_data = create_setops()
sample_schema = define_schema()
rdd = spark.sparkContext.parallelize(sample_data)
df = spark.createDataFrame(rdd, sample_schema)
#df.show(10)

[['ST_J_0999990', 1.0, 1000], ['ST_J_0999991', 1.0, 1000], ['ST_J_0999992', 1.0, 1000], ['ST_J_0999993', 1.0, 1000], ['ST_J_0999994', 1.0, 1000], ['ST_J_0999995', 1.0, 1000], ['ST_J_0999996', 1.0, 1000], ['ST_J_0999997', 1.0, 1000], ['ST_J_0999998', 1.0, 1000], ['ST_J_0999999', 1.0, 1000]]
CPU times: user 18.2 s, sys: 1.4 s, total: 19.6 s
Wall time: 22.2 s


In [7]:
%%time 
# HDFS 에 /user/root/inven/setop 폴더와 파일이 생성 됨.  
tbl_setop_name = f'inven/table-set-{scale}'
file_format = 'parquet'
df.write.save(path=tbl_setop_name, format=file_format, mode='append', sep=',')

CPU times: user 7.87 ms, sys: 4.65 ms, total: 12.5 ms
Wall time: 32.1 s


In [8]:
%%time 
# 기록한 파일 다시 읽어 들이기 
# 저장 결과 확인하기 
lines = spark.read.format(file_format).option('path', tbl_setop_name).load()
data_count = lines.count()
print(f'DATA Count : {data_count:,}')
lines.show(5)

DATA Count : 10,000,000
+------------+--------+-------+
|       setop|inv_rate|inv_val|
+------------+--------+-------+
|ST_C_0499584|     1.0|   1000|
|ST_C_0499585|     1.0|   1000|
|ST_C_0499586|     1.0|   1000|
|ST_C_0499587|     1.0|   1000|
|ST_C_0499588|     1.0|   1000|
+------------+--------+-------+
only showing top 5 rows

CPU times: user 3.3 ms, sys: 3.63 ms, total: 6.93 ms
Wall time: 2.6 s


In [10]:
spark.stop()